In [4]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

%load_ext autoreload
%autoreload 2

import pandas as pd

from pathlib import Path
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
path = '/mnt/c/Users/rwmas/GitHub/xai/python-asd-copy/xai/test/rs_knockoff_covid/'

files = Path(path).glob('*.csv')  

dfs = list()
for f in files:
    data = pd.read_csv(f, sep=';')
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)

In [14]:
### We have done 69 tests on the covid dataset, each test with 20,000 iterations.
### This notebooks compares the results of each iteration to see if the ranks differe among various iterations or not.
### To varify that, we took combine all the files and took the mean rank of each column and divide it by the max rank of that column among all the files/test. The highest rank
### /relevance of a col in each test is 40 since there are 40 number of columns in covid dataset. The rank number than descend all the way to 0 where 0 indicates no 
# contribution/relevance at all. The idea is that if the mean of a column will give us the mean rank of the respective variable, i.e., a mean of 40 will show that this column
# was always ranked in 40 in all the files/tests but if a column would have ranked 39 in even one test but 40 in all other rests, its mean would be less than 40. 
# Now if we divide this mean with the max rank we will be able to identify columns where the rank was varied among the 69 test. I.e., mean_rank / max_rank < 1.

In [72]:
df_max_ranks = df.groupby(['cols'])['0'].max()
df_mean_ranks = df.groupby(['cols'])['0'].mean()

In [70]:
df_final = pd.merge(df_max_ranks, df_mean_ranks, on='cols').reset_index()
df_final.columns = ['cols', 'max_ranked', 'mean_ranked']

In [74]:
df_final = df_final.sort_values(by=['mean_ranked'], ascending=False)
df_final

,cols,max_ranked,mean_ranked
38,new_cases_per_million,40.0,40.0
0,Acute hepatitis,39.0,39.0
2,Alzheimer's disease and other dementias,38.0,38.0
13,Drowning,37.0,37.0
30,Protein-energy malnutrition,36.0,36.0
39,population_cov,35.0,35.0
5,Chronic kidney disease,34.0,34.0
26,Nutritional deficiencies,33.0,33.0
8,Conflict and terrorism,32.0,32.0
9,Density_km2,31.0,31.0


In [ ]:
# since none of the df_final.mean_ranked/df_final.max_ranked values are < 1, this implies that all our knockoff iterations gives the same results. 
# As per the above dataset new_cases_per_million is of highest importance and all those with 0 rank has no relevance at all.
df_final.mean_ranked/df_final.max_ranked